In [1]:
test_file = "10_cols_1_mb"
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LargeRowBenchmark").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/04 09:48:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.parquet(f"data/dummy/{test_file}.parquet")
df.createOrReplaceTempView("T")
df.show(1)

+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|uid|               col_0|               col_1|               col_2|               col_3|               col_4|               col_5|               col_6|               col_7|               col_8|               col_9|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  1|This is a sentenc...|This is a sentenc...|This is a sentenc...|This is a sentenc...|This is a sentenc...|This is a sentenc...|This is a sentenc...|This is a sentenc...|This is a sentenc...|This is a sentenc...|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----

In [3]:
# print schema, and length of string columns
from pyspark.sql.types import StringType
from pyspark.sql.functions import length, col

# show string column length
for c in df.columns:
    if df.schema[c].dataType is StringType():
        str_length = df.select(length(col(c))).collect()[0][0]
        print(f"Column: {c}, {df.schema[c].dataType}, length: {str_length}")
    else:
        print(f"Column: {c}, {df.schema[c].dataType}")

Column: uid, LongType()
Column: col_0, StringType(), length: 1048576
Column: col_1, StringType(), length: 1048576
Column: col_2, StringType(), length: 1048576
Column: col_3, StringType(), length: 1048576
Column: col_4, StringType(), length: 1048576
Column: col_5, StringType(), length: 1048576
Column: col_6, StringType(), length: 1048576
Column: col_7, StringType(), length: 1048576
Column: col_8, StringType(), length: 1048576
Column: col_9, StringType(), length: 1048576


24/10/04 09:48:50 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
sql = "select length(col_0)/1024/1024 as size_mb from T"
spark.sql(sql).show()

+-------+
|size_mb|
+-------+
|    1.0|
+-------+



In [7]:
spark.stop()